In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
import winsound
RESOURCES_FOLDER = 'resources'
FAST_TEXT_MODEL = f'{RESOURCES_FOLDER}/lid.176.bin'
DATASET_PATH = f'{RESOURCES_FOLDER}/dataset.csv'
RANDOM_STATE = 1234
PREPROCESSED_DATASET = f'{RESOURCES_FOLDER}/preprocessed_dataset.csv'

# DA AGGIUNGERE
import pickle
model_path_builder = lambda model_name: f'{RESOURCES_FOLDER}/models/{model_name}.pickle'

# 

def ping():
    winsound.PlaySound("WindowsUnlock", winsound.SND_ALIAS)

dataset = pd.read_csv(PREPROCESSED_DATASET)


ValueError: Length mismatch: Expected axis has 2 elements, new values have 3 elements

In [43]:
dataset = dataset[dataset['stars'] != 3]
dataset.loc[(dataset['stars'] == 1) | (dataset['stars'] == 2), 'stars'] = 0
dataset.loc[(dataset['stars'] == 4) | (dataset['stars'] == 5), 'stars'] = 1

In [44]:
dataset.stars.value_counts()

stars
0    180000
1    180000
Name: count, dtype: int64

In [19]:
X = dataset.text
y = dataset.stars - 1

In [4]:
dataset = pd.read_csv('resources/dataset.csv')
dataset.columns = ['stars', 'title', 'text']

In [5]:
dataset['text'] = dataset['title'] + " " + dataset['text']

In [8]:
import re
def normalise(texts):
    return [re.compile(r"[^a-z0-1\s]").sub(r" ",re.compile(r"['\W']").sub(r" ",text.lower())) for text in texts]

dataset['text'] = dataset['text'].apply(normalise)

TypeError: 'float' object is not iterable

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    stratify=y, 
                                                    random_state=RANDOM_STATE)
print(f'Train size: {len(train_y)}')
print(f'Test size: {len(test_y)}')

In [21]:
vectorizer = TfidfVectorizer().fit(train_x)
train_features = vectorizer.transform(train_x)
test_features = vectorizer.transform(test_x)

In [22]:
model = SVC()
grid_params = {'C': uniform(loc=0, scale=1), 'kernel': ['poly', 'rbf', 'sigmoid']}

In [23]:
clf = RandomizedSearchCV(model, 
                         grid_params,
                         scoring='accuracy',
                         cv=5,
                         n_iter=3,
                         n_jobs=-1,
                         verbose=3)

best_svm_model = clf.fit(train_features, train_y)
ping()
ping()

Fitting 5 folds for each of 3 candidates, totalling 15 fits


KeyboardInterrupt: 

In [ ]:
print(f'Best Accuracy Score: {round(best_svm_model.best_score_, 3)}')
print(f'Best Hyperparameters: {best_svm_model.best_params_}')

In [ ]:
with open(model_path_builder('SVC'), 'wb') as f:
      pickle.dump(best_svm_model, f)

In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import models,layers,optimizers
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re
%matplotlib inline
import os

In [46]:
train_x, test_x, train_y, test_y = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    stratify=y, 
                                                    random_state=RANDOM_STATE)
train_x, validation_x, train_y, validation_y = train_test_split(train_x, train_y, 
                                                    test_size=0.2, 
                                                    stratify=train_y, 
                                                    random_state=RANDOM_STATE)

In [47]:
tokenizer=Tokenizer(num_words=12000)
tokenizer.fit_on_texts(train_x)
train_texts = tokenizer.texts_to_sequences(train_x)
val_texts = tokenizer.texts_to_sequences(validation_x)
test_texts = tokenizer.texts_to_sequences(test_x)

In [48]:
mlen = max(len(train_ex) for train_ex in train_texts)
train_texts = pad_sequences(train_texts, maxlen=mlen)
val_texts = pad_sequences(val_texts, maxlen=mlen)
test_texts = pad_sequences(test_texts, maxlen=mlen)

In [49]:
from keras.src.utils import to_categorical

# train_y = to_categorical(train_y, num_classes=2)
# validation_y = to_categorical(validation_y, num_classes=5)
# test_y = to_categorical(test_y, num_classes=5)


In [50]:
def build_model():
    sequences = layers.Input(shape=(mlen,))
    embedded = layers.Embedding(12000, 64)(sequences)
    x = layers.Conv1D(252, 3, activation='relu')(embedded)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(3)(x)
    x = layers.Conv1D(128, 5, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(5)(x)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = layers.GlobalMaxPool1D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [51]:
from keras.src.callbacks import EarlyStopping

with tf.device('/GPU:0'):
    model = build_model()
    model.fit(train_texts,train_y,
              batch_size=128,
              epochs=100,
              validation_data=(val_texts, validation_y),
              callbacks=[EarlyStopping(monitor='val_loss', patience=10, verbose=1)]
)

Epoch 1/100
100/100 [==============================] - 13s 111ms/step - loss: -1381.9840 - accuracy: 0.4959 - val_loss: -402.4581 - val_accuracy: 0.5000
Epoch 2/100
 10/100 [==>...........................] - ETA: 9s - loss: -8562.0615 - accuracy: 0.5008

KeyboardInterrupt: 

In [ ]:
preds = model.predict(test_texts)
print('Accuracy score: {:0.4}'.format(accuracy_score(test_y, 1 * (preds > 0.5))))
print('F1 score: {:0.4}'.format(f1_score(test_y, 1 * (preds > 0.5))))
print('ROC AUC score: {:0.4}'.format(roc_auc_score(test_y, preds)))

In [ ]:
from keras.src.optimizers import Adam
from keras import layers, models


def build_model():
    sequences = layers.Input(shape=(mlen,))
    embedded = layers.Embedding(12000, 64, input_length=mlen)(sequences)
    x = layers.Conv1D(252, 3, activation='relu')(embedded)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(3)(x)
    x = layers.Conv1D(128, 5, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(5)(x)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = layers.GlobalMaxPool1D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_model()

# Training
epochs = 10
batch_size = 128

history = model.fit(
    train_texts, train_y,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(val_texts, validation_y)
)

# Evaluation
loss, accuracy = model.evaluate(test_texts, test_y)

# Training and evaluation code
